# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import requests 
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('/Users/malamapono/JupyterProjects/AlgorithmicTrading/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN
stocks.head()


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 111699012,
 'calculationPrice': 'close',
 'change': -0.66,
 'changePercent': -0.00363,
 'close': 0,
 'closeSource': 'cfiliofa',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 184.71,
 'iexCloseTime': 1695455564048,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 185.384,
 'iexOpenTime': 1713252687122,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1660858258707,
 'latestPrice': 179.06,
 'latestSource': 'Close',
 'latestTime': 'December 31, 2021',
 'latestUpdate': 1663504323538,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 2930646520974,
 'odd

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_df = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        price = data[symbol]['quote']['latestPrice']
        pe_ratio = data[symbol]['quote']['peRatio']
        final_df = final_df.append(pd.Series([symbol,price,pe_ratio,'NA'],index=my_columns),ignore_index=True)
    
final_df.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,159.59,50.46,NA
1,AAL,18.56,-3.65,NA
2,AAP,241.48,24.48,NA
3,AAPL,185.06,16.25,NA
4,ABBV,140.70,33.06,NA


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df['Price-to-Earnings Ratio'] = final_df['Price-to-Earnings Ratio'].astype('float64')
final_df = final_df.nsmallest(50,'Price-to-Earnings Ratio')
final_df.reset_index(inplace=True)
final_df.drop('index',axis=1,inplace=True)

In [8]:
final_df.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,HPE,15.83,3.11,NA
1,BIO,764.05,3.48,NA
2,HPQ,38.69,3.60,NA
3,EBAY,68.90,3.65,NA
4,COO,434.87,3.66,NA


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
portfolio_size = float(input("Enter the value of your portfolio:"))

Enter the value of your portfolio:100000


Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = portfolio_size/50
final_df['Number of Shares to Buy'] = position_size/final_df['Price']
final_df.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,HPE,15.83,3.11,126.342388
1,BIO,764.05,3.48,2.617630
2,HPQ,38.69,3.60,51.692944
3,EBAY,68.90,3.65,29.027576
4,COO,434.87,3.66,4.599076


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

pe_ratio = data[symbol]['quote']['peRatio']

pb_ratio = data[symbol]['advanced-stats']['priceToBook']

ps_ratio = data[symbol]['advanced-stats']['priceToSales']

enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_ebitda = enterprise_value/ebitda

gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_gp = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        price = data[symbol]['quote']['latestPrice']
        pe_ratio = data[symbol]['quote']['peRatio']

        pb_ratio = data[symbol]['advanced-stats']['priceToBook']

        ps_ratio = data[symbol]['advanced-stats']['priceToSales']

        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        try:
            ev_ebitda = enterprise_value/ebitda
        except:
            ev_ebitda = np.nan

        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_gp = enterprise_value/gross_profit
        except:
            ev_gp = np.nan
        rv_df = rv_df.append(pd.Series([symbol,price,'NA',pe_ratio,'NA',pb_ratio,'NA',ps_ratio,'NA',ev_ebitda,'NA',ev_gp,'NA','NA'],index=rv_columns),ignore_index=True)
        
rv_df.head()    

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,162.70,NA,52.08,NA,10.07,NA,8.21,NA,31.638790,NA,14.822453,NA,NA
1,AAL,18.40,NA,-3.68,NA,-1.62,NA,0.498,NA,-7.344490,NA,1.451998,NA,NA
2,AAP,241.09,NA,24.72,NA,4.91,NA,1.43,NA,13.933275,NA,3.229183,NA,NA
3,AAPL,181.47,NA,15.88,NA,23.98,NA,4.09,NA,12.884093,NA,10.019763,NA,NA
4,ABBV,137.20,NA,33.22,NA,18.2,NA,4.48,NA,11.547189,NA,8.377406,NA,NA


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
is_NaN = rv_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rv_df[row_has_NaN]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,188.400,NA,62.31,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
40,AON,308.440,NA,None,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
71,BRK.B,306.000,NA,None,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
118,CTL,11.000,NA,9.92,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
136,DISCK,23.800,NA,None,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
165,ETFC,49.340,NA,14.7,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
186,FLIR,57.700,NA,32.91,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
190,FOX,35.010,NA,None,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
204,GOOG,3017.140,NA,None,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA
325,MXIM,103.290,NA,35.09,NA,None,NA,None,NA,NaN,NA,NaN,NA,NA


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price',
    'Price-to-Earnings Ratio',
    'Price-to-Book Ratio',
    'Price-to-Sales Ratio',
    'EV/EBITDA',
    'EV/GP']:
    rv_df[column] = rv_df[column].fillna(rv_df[column].mean())

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
is_NaN = rv_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rv_df[row_has_NaN]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [16]:
rv_df.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,162.70,NA,52.08,NA,10.07,NA,8.210,NA,31.638790,NA,14.822453,NA,NA
1,AAL,18.40,NA,-3.68,NA,-1.62,NA,0.498,NA,-7.344490,NA,1.451998,NA,NA
2,AAP,241.09,NA,24.72,NA,4.91,NA,1.430,NA,13.933275,NA,3.229183,NA,NA
3,AAPL,181.47,NA,15.88,NA,23.98,NA,4.090,NA,12.884093,NA,10.019763,NA,NA
4,ABBV,137.20,NA,33.22,NA,18.20,NA,4.480,NA,11.547189,NA,8.377406,NA,NA


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
old_columns = ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']
new_columns = ['PE Percentile','PB Percentile','PS Percentile','EV/EBITDA Percentile','EV/GP Percentile',]
for i in range(len(old_columns)):
    old_column = old_columns[i]
    new_column = new_columns[i]
    rv_df[new_column] = rv_df[old_column].apply(lambda x:score(rv_df[old_column],x)/100)
    
rv_df.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,162.70,NA,52.08,0.827723,10.07,0.805941,8.210,0.831683,31.638790,0.843564,14.822453,0.835644,NA
1,AAL,18.40,NA,-3.68,0.053465,-1.62,0.055446,0.498,0.051485,-7.344490,0.023762,1.451998,0.063366,NA
2,AAP,241.09,NA,24.72,0.499010,4.91,0.617822,1.430,0.206931,13.933275,0.451485,3.229183,0.180198,NA
3,AAPL,181.47,NA,15.88,0.299010,23.98,0.938614,4.090,0.591089,12.884093,0.411881,10.019763,0.675248,NA
4,ABBV,137.20,NA,33.22,0.653465,18.20,0.916832,4.480,0.622772,11.547189,0.352475,8.377406,0.584158,NA


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
rv_df['RV Score'] = rv_df[new_columns].mean(axis=1)

In [19]:
rv_df.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,162.70,NA,52.08,0.827723,10.07,0.805941,8.210,0.831683,31.638790,0.843564,14.822453,0.835644,0.828911
1,AAL,18.40,NA,-3.68,0.053465,-1.62,0.055446,0.498,0.051485,-7.344490,0.023762,1.451998,0.063366,0.049505
2,AAP,241.09,NA,24.72,0.499010,4.91,0.617822,1.430,0.206931,13.933275,0.451485,3.229183,0.180198,0.391089
3,AAPL,181.47,NA,15.88,0.299010,23.98,0.938614,4.090,0.591089,12.884093,0.411881,10.019763,0.675248,0.583168
4,ABBV,137.20,NA,33.22,0.653465,18.20,0.916832,4.480,0.622772,11.547189,0.352475,8.377406,0.584158,0.625941


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_df = rv_df.nsmallest(50,'RV Score')
rv_df.reset_index(inplace=True)
rv_df.drop('index',axis=1,inplace=True)

In [21]:
rv_df.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,24.87,NA,6.37,0.097030,0.4719,0.057426,0.3929,0.031683,3.093006,0.031683,0.388193,0.007921,0.045149
1,AAL,18.40,NA,-3.68,0.053465,-1.6200,0.055446,0.4980,0.051485,-7.344490,0.023762,1.451998,0.063366,0.049505
2,HPQ,39.47,NA,3.67,0.069307,-12.9000,0.037624,0.3317,0.015842,3.975768,0.051485,1.709073,0.083168,0.051485
3,PRU,109.04,NA,6.13,0.091089,0.6746,0.067327,0.6100,0.055446,4.562549,0.063366,0.564287,0.011881,0.057822
4,BEN,34.16,NA,4.85,0.077228,0.7772,0.079208,1.0440,0.130693,3.167047,0.033663,0.853745,0.019802,0.068119


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [22]:
rv_df['Number of Shares to Buy'] = position_size/rv_df['Price']
rv_df.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,24.87,80.418175,6.37,0.097030,0.4719,0.057426,0.3929,0.031683,3.093006,0.031683,0.388193,0.007921,0.045149
1,AAL,18.40,108.695652,-3.68,0.053465,-1.6200,0.055446,0.4980,0.051485,-7.344490,0.023762,1.451998,0.063366,0.049505
2,HPQ,39.47,50.671396,3.67,0.069307,-12.9000,0.037624,0.3317,0.015842,3.975768,0.051485,1.709073,0.083168,0.051485
3,PRU,109.04,18.341893,6.13,0.091089,0.6746,0.067327,0.6100,0.055446,4.562549,0.063366,0.564287,0.011881,0.057822
4,BEN,34.16,58.548009,4.85,0.077228,0.7772,0.079208,1.0440,0.130693,3.167047,0.033663,0.853745,0.019802,0.068119


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx',engine='xlsxwriter')
rv_df.to_excel(writer,sheet_name='Value Strategy',index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.000000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', float_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [26]:
writer.save()